## Totals and subtotals in pandas *DataFrame*s

In [80]:
# Generate the data
import pandas as pd
import numpy as np

np.random.seed(42)

df = pd.DataFrame({
"Group": 4*['X']+6*['Y'],
"Subgroup": 3*['a']+['z']+3*['a']+3*['z'],
"Value": np.random.randint(20, size=10)
})

In [81]:
df

,Group,Subgroup,Value
0,X,a,6
1,X,a,19
2,X,a,14
3,X,z,10
4,Y,a,7
5,Y,a,6
6,Y,a,18
7,Y,z,10
8,Y,z,10
9,Y,z,3


### 1. Adding a "Total" row

In [82]:
# Two ways to create the same dataframe as above,
# but with the feature "Group" set to "Total" for each observations

## The functional way :
df.assign(Group=lambda x: "Total")

,Group,Subgroup,Value
0,Total,a,6
1,Total,a,19
2,Total,a,14
3,Total,z,10
4,Total,a,7
5,Total,a,6
6,Total,a,18
7,Total,z,10
8,Total,z,10
9,Total,z,3


In [83]:
## A second method :
df.assign(Group=["Total"]*df.shape[0])

,Group,Subgroup,Value
0,Total,a,6
1,Total,a,19
2,Total,a,14
3,Total,z,10
4,Total,a,7
5,Total,a,6
6,Total,a,18
7,Total,z,10
8,Total,z,10
9,Total,z,3


In [84]:
# Concatenate the dataframes
df_total = pd.concat([df, df.assign(Group=lambda x: "Total")])

In [85]:
# Aggregate
df_total.groupby('Group').mean()

,Value
Group,
Total,10.30
X,12.25
Y,9.00


**Oh no !** The total sould be at the bottom. To achieve that, the "Group" variable in the concatenated *DataFrame* must be cast to a *dtype* "category". More specifically, it must be an ordered category, with the level "Total" in final position. More on categorical data with pandas [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html).

In [86]:
# Cast "Group" to a category with ordered levels
from pandas.api.types import CategoricalDtype
categorical_type = CategoricalDtype(categories=list(df['Group'].unique()) + ['Total'], ordered=True)
df_total['Group'] = df_total['Group'].astype(categorical_type)

In [87]:
# Now we're happy :)
df_total.groupby('Group').mean()

,Value
Group,
X,12.25
Y,9.00
Total,10.30


### 2. Adding total and subtotals rows

In [88]:
# Concatenate
df_subtotal = pd.concat([
    df,
    df.assign(Subgroup=lambda x: "Total"),
    df.assign(Group=lambda x: "Total", Subgroup=lambda x: "Total")
])

# Order levels and cast grouping variables to dtype category
cat_Group = CategoricalDtype(categories=list(df['Group'].unique()) + ['Total'], ordered=True)
cat_Subgroup = CategoricalDtype(categories=list(df['Subgroup'].unique()) + ['Total'], ordered=True)
df_subtotal['Group'] = df_subtotal['Group'].astype(cat_Group)
df_subtotal['Subgroup'] = df_subtotal['Subgroup'].astype(cat_Subgroup)

In [89]:
# Aggregate
df_subtotal.groupby(by=['Group', 'Subgroup'], observed=True).mean()

Value
Group Subgroup           
X     a         13.000000
      z         10.000000
      Total     12.250000
Y     a         10.333333
      z          7.666667
      Total      9.000000
Total Total     10.300000

Hooray !